In [25]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [26]:
from dotenv import load_dotenv

load_dotenv()

False

In [27]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [28]:
books.shape

(5197, 13)

In [29]:


books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [32]:
import polars as pl
books = pl.from_pandas(books)

In [34]:
books.head(3)

isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
i64,str,str,str,str,str,str,f64,f64,f64,f64,str,str
9780002005883,"""0002005883""","""Gilead""","""Marilynne Robinson""","""Fiction""","""http://books.google.com/books/…","""A NOVEL THAT READERS and criti…",2004.0,3.85,247.0,361.0,"""Gilead""","""9780002005883 A NOVEL THAT REA…"
9780002261982,"""0002261987""","""Spider's Web""","""Charles Osborne;Agatha Christi…","""Detective and mystery stories""","""http://books.google.com/books/…","""A new 'Christie for Christmas'…",2000.0,3.83,241.0,5164.0,"""Spider's Web: A Novel""","""9780002261982 A new 'Christie …"
9780006178736,"""0006178731""","""Rage of angels""","""Sidney Sheldon""","""Fiction""","""http://books.google.com/books/…","""A memorable, mesmerizing heroi…",1993.0,3.93,512.0,29532.0,"""Rage of angels""","""9780006178736 A memorable, mes…"


In [35]:
books.select("tagged_description").write_csv(
    "tagged_description.txt",
    separator="\n",
    include_header=False
)

In [36]:
# books["tagged_description"].to_csv("tagged_description.txt",
#                                    sep = "\n",
#                                    index = False,
#                                    header = False)

In [46]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=150, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 800
Created a chunk of size 1214, which is longer than the specified 800
Created a chunk of size 960, which is longer than the specified 800
Created a chunk of size 843, which is longer than the specified 800
Created a chunk of size 881, which is longer than the specified 800
Created a chunk of size 1088, which is longer than the specified 800
Created a chunk of size 1189, which is longer than the specified 800
Created a chunk of size 1267, which is longer than the specified 800
Created a chunk of size 887, which is longer than the specified 800
Created a chunk of size 2010, which is longer than the specified 800
Created a chunk of size 1225, which is longer than the specified 800
Created a chunk of size 1184, which is longer than the specified 800
Created a chunk of size 1214, which is longer than the specified 800
Created a chunk of size 1191, which is longer than the specified 800
Created a chunk of size 1057, which is

In [47]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [48]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [49]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"  # Лучшее качество
)


In [50]:
# Использование:
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings
)

In [7]:
# db_books = Chroma.from_documents(
#     documents,
#     embedding=OpenAIEmbeddings())

In [51]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='9c19b993-1b56-41dd-b366-b703f5e8b540', metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson v

In [54]:
books= books.to_pandas()

In [55]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

ValueError: invalid literal for int() with base 10: '"9780067575208'

In [56]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [57]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...
270,9780060885373,0060885378,Little House in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=7JctN...,A year in the life of two young girls growing ...,2007.0,4.18,198.0,193862.0,Little House in the Big Woods,9780060885373 A year in the life of two young ...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
427,9780064434874,0064434877,Christmas in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."
428,9780064434942,006443494X,A Little House Birthday,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=fyfDR...,Join the Ingalls family as they celebrate litt...,1998.0,4.13,40.0,444.0,A Little House Birthday,9780064434942 Join the Ingalls family as they ...
430,9780064435260,0064435261,A Little Prairie House,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=pRSju...,"Long, long ago, a little girl named Laura Inga...",1999.0,4.19,32.0,1533.0,A Little Prairie House,"9780064435260 Long, long ago, a little girl na..."
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
692,9780140448009,0140448004,Three Tales,Gustave Flaubert;Roger Whitehouse;Geoffrey Wall,Fiction,http://books.google.com/books/content?id=XFzga...,Features short fiction by the French naturalis...,2005.0,3.71,110.0,3050.0,Three Tales,9780140448009 Features short fiction by the Fr...
